In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install dependencies
!pip install -U albumentations

import os
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import transforms, datasets, models
import albumentations as A; print(A.__version__)
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2.0.8


In [ ]:
# Albumentations augmentations
train_aug = A.Compose([
    A.RandomRotate90(),
    A.HorizontalFlip(),
    A.Affine(translate_percent={"x":0.1, "y":0.1}, scale=(0.9, 1.1), rotate=(-30, 30)),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
    A.GaussNoise(var_limit=(10.0, 50.0)),
    A.Resize(224, 224),
    A.Normalize(),  # This line is essential!
    ToTensorV2()
])

val_aug = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),  # Add normalization here too
    ToTensorV2()
])


class ThermalDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform):
        self.dataset = datasets.ImageFolder(root)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        img = np.array(img)
        img = self.transform(image=img)['image']
        return img, label

# DataLoaders
train_ds = ThermalDataset('/content/drive/MyDrive/DATASET/train', train_aug)
val_ds   = ThermalDataset('/content/drive/MyDrive/DATASET/val', val_aug)
test_ds  = ThermalDataset('/content/drive/MyDrive/DATASET/test', val_aug)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=2)

/tmp/ipython-input-16-654509002.py:7: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0)),


In [ ]:
# Load MobileNetV2 pretrained on ImageNet
model = models.mobilenet_v2(pretrained=True)
# Replace classifier
in_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(in_features, 2)
)
model = model.to('cuda')

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

def train_one_epoch():
    model.train()
    total_loss, correct = 0, 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss/len(train_ds), correct/len(train_ds)

def evaluate(loader):
    model.eval()
    loss, correct = 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.cuda(), labels.cuda()
            outputs = model(imgs)
            loss += criterion(outputs, labels).item() * imgs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
    return loss/len(loader.dataset), correct/len(loader.dataset)

# Training loop
best_acc = 0
for epoch in range(1, 41):
    train_loss, train_acc = train_one_epoch()
    val_loss, val_acc = evaluate(val_loader)
    scheduler.step()
    print(f"Epoch {epoch}: Train Acc {train_acc:.4f}, Val Acc {val_acc:.4f}")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'mobilenetv2_best.pth')

Epoch 1: Train Acc 0.8337, Val Acc 0.9169
Epoch 2: Train Acc 0.9216, Val Acc 0.9340
Epoch 3: Train Acc 0.9414, Val Acc 0.9652
Epoch 4: Train Acc 0.9544, Val Acc 0.9407
Epoch 5: Train Acc 0.9582, Val Acc 0.9676
Epoch 6: Train Acc 0.9624, Val Acc 0.9688
Epoch 7: Train Acc 0.9670, Val Acc 0.9413
Epoch 8: Train Acc 0.9690, Val Acc 0.9639
Epoch 9: Train Acc 0.9738, Val Acc 0.9707
Epoch 10: Train Acc 0.9755, Val Acc 0.9756
Epoch 11: Train Acc 0.9748, Val Acc 0.9590
Epoch 12: Train Acc 0.9825, Val Acc 0.9743
Epoch 13: Train Acc 0.9802, Val Acc 0.9780
Epoch 14: Train Acc 0.9844, Val Acc 0.9762
Epoch 15: Train Acc 0.9823, Val Acc 0.9762
Epoch 16: Train Acc 0.9857, Val Acc 0.9823
Epoch 17: Train Acc 0.9892, Val Acc 0.9811
Epoch 18: Train Acc 0.9912, Val Acc 0.9811
Epoch 19: Train Acc 0.9893, Val Acc 0.9817
Epoch 20: Train Acc 0.9902, Val Acc 0.9811
Epoch 21: Train Acc 0.9908, Val Acc 0.9804
Epoch 22: Train Acc 0.9924, Val Acc 0.9817
Epoch 23: Train Acc 0.9910, Val Acc 0.9811
Epoch 24: Train Acc 

In [ ]:
model.load_state_dict(torch.load('mobilenetv2_best.pth'))
test_loss, test_acc = evaluate(test_loader)
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.9975


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.cuda(), labels.cuda()
        outputs = model(imgs)
        preds = outputs.argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

# Classification report
target_names = ['Female', 'Male']
print(classification_report(all_labels, all_preds, target_names=target_names))


Confusion Matrix:
 [[394   1]
 [  1 414]]
              precision    recall  f1-score   support

      Female       1.00      1.00      1.00       395
        Male       1.00      1.00      1.00       415

    accuracy                           1.00       810
   macro avg       1.00      1.00      1.00       810
weighted avg       1.00      1.00      1.00       810

